$$Machine Learning$$

$Selecting Data for Modeling$

Import pandas and read the data from the csv file.Use dot notation to select the prediction target and use a column list to select the necessary features.

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
df=pd.read_csv("all_perth_310121.csv")
df.head()
df.columns

Index(['ADDRESS', 'SUBURB', 'PRICE', 'BEDROOMS', 'BATHROOMS', 'GARAGE',
       'LAND_AREA', 'FLOOR_AREA', 'BUILD_YEAR', 'CBD_DIST', 'NEAREST_STN',
       'NEAREST_STN_DIST', 'DATE_SOLD', 'POSTCODE', 'LATITUDE', 'LONGITUDE',
       'NEAREST_SCH', 'NEAREST_SCH_DIST', 'NEAREST_SCH_RANK'],
      dtype='object')

Select the column we want to predict and assign it(by convention) to y(the prediction target).

In [24]:
y=df.PRICE#y is the target variable

$Selecting Features$

The columns that are inputted into our model (and later used to make predictions) are called "features." In our case, those would be the columns used to determine the home price. Sometimes, you will use all columns except the target as features. Other times you'll be better off with fewer features.

We select multiple features by providing a list of column names inside brackets. Each item in that list should be a string (with quotes).By convention, this data is called X.

In [36]:
x=df[['BEDROOMS', 'BATHROOMS','LAND_AREA','LATITUDE','LONGITUDE']]
x.describe()

,BEDROOMS,BATHROOMS,LAND_AREA,LATITUDE,LONGITUDE
count,33656.000000,33656.000000,33656.000000,33656.000000,33656.000000
mean,3.659110,1.823063,2740.644016,-31.960664,115.879265
std,0.752038,0.587427,16693.513215,0.177780,0.118137
min,1.000000,1.000000,61.000000,-32.472979,115.582730
25%,3.000000,1.000000,503.000000,-32.068437,115.789763
50%,4.000000,2.000000,682.000000,-31.933231,115.854198
75%,4.000000,2.000000,838.000000,-31.843818,115.970722
max,10.000000,16.000000,999999.000000,-31.457450,116.343201


$Building Your Model$

We use scikit-learn(sklearn) library to create models.It is the most popular library for modeling the types of data typically stored in DataFrames.The steps to building and using a model are:

- Define:
    What type of model will it be? A decision tree? Some other type of model? Some other parameters of the model type are specified too.
- Fit: 
    Capture patterns from provided data. This is the heart of modeling.
- Predict: 
    Just what it sounds like
- Evaluate:
    Determine how accurate the model's predictions are.

In [38]:
from sklearn.tree import DecisionTreeRegressor

price_model=DecisionTreeRegressor(random_state=1)#random_state is used to get the same result every time we run the code,ignoring the randomness involved when choosing the feature for splitting into two leaves.
price_model.fit(x,y);#captures patterns

We now have a fitted model that we can use to make predictions.In practice, you'll want to make predictions for new data rather than the data we already know. But we'll make predictions for the first few rows of the training data to see how the predict function works.

In [48]:
print("Prediction data")
print(x.head())
print(df.iloc[:5]['PRICE'])
print(price_model.predict(x.head()))

Prediction data
   BEDROOMS  BATHROOMS  LAND_AREA   LATITUDE   LONGITUDE
0         4          2        600 -32.115900  115.842450
1         3          2        351 -32.193470  115.859554
2         3          1        719 -32.120578  115.993579
3         2          1        651 -31.900547  116.038009
4         4          1        466 -31.885790  115.947780
0    565000
1    365000
2    287000
3    255000
4    325000
Name: PRICE, dtype: int64
[565000. 365000. 287000. 255000. 325000.]


$Model$ $Validation$

You'll need to check if your model is good or not for almost all models you create. Usually, the best way to measure if a model is good is to see if its predictions match what really happens. This is called predictive accuracy.

First,we find error via the Mean Absolute Error(MAE) metric. The MAE is the average of the absolute differences between predictions and actual values. It gives you an idea of how wrong the predictions were.

In [49]:
from sklearn.metrics import mean_absolute_error

predicted_prices=price_model.predict(x)
mean_absolute_error(y,predicted_prices)

104.80180849378019

BUT,THIS IS WRONG!!
The score we got is from using the same houses to make and test the model, which can be misleading.

For example, our model might learn from our specific data that green doors mean expensive houses, making it seem accurate. But in reality, door color doesn't affect house price.

The scikit-learn library has a function train_test_split to break up the data into two pieces. We'll use some of that data as training data to fit the model, and we'll use the other data as validation data to calculate mean_absolute_error.
So, we test the model on new, unseen data, called validation data, to see how well it really works.


In [50]:
from sklearn.model_selection import train_test_split

train_X,val_X,train_y,val_y=train_test_split(x,y,random_state=0)
new_price_model=DecisionTreeRegressor()
new_price_model.fit(train_X,train_y)
val_predictions=new_price_model.predict(val_X)
print(mean_absolute_error(val_y,val_predictions))


147037.874613739


$Underfitting$ $and$ $Overfitting$